In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud ,STOPWORDS
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize 

import re
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation,Flatten
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/output/Final_Unique_Tweets.csv")

In [ ]:
df.shape

(430187, 29)

In [ ]:
#Remove null values
df = df.dropna(axis=0, subset=['Text'])

In [ ]:
df.shape

(430187, 29)

In [ ]:
#Remove null values
df=df.drop_duplicates(subset='Text', keep="first")

In [ ]:
df.shape

(430187, 29)

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Preprocessing

In [ ]:
def replaceDegfanged(text):
    """ Replaces url address with "url" """
    text = re.sub('([A-Za-z]+://)([-\w]+(?:\.\w[-\w]*)+)(:\d+)?(/[^.!,?;"\x27<>()\[\]{}\s\x7F-\xFF]*(?:[.!,?]+[^.!,?;"\x27<>()\[\]{}\s\x7F-\xFF]+)*)?','[defanged]',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

In [ ]:
def replaceURL(text):
    """ Replaces url address with "url" """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','[url]',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('A'):
        return wordnet.ADJ
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    elif treebank_tag.startswith('SYM'):
        return wordnet.SYM
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV    
    elif treebank_tag.startswith('WRB'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
#@title
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [ ]:
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()

https://www.kaggle.com/code/christofhenkel/how-to-preprocessing-for-glove-part1-eda/notebook
https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing/notebook



In [ ]:

def preprocessing_df(text):
  tweets = []
  for sentence in tqdm(text):
    sentence = sentence.lower() # converting the words to lower case
    sentence = re.sub("@[A-Za-z0-9_]+","", sentence) # removing the words starts with "@"
    #sentence = re.sub("#[A-Za-z0-9_]+","", sentence) # removing the words starts with "H"
    sentence = re.sub(r'https://t.co/\S+','', sentence)
    sentence =  replaceURL(sentence) # replace the url from the sentence
    sentence = re.sub(r'hxxp\S+','[defanged]', sentence)
    sentence = replaceDegfanged(sentence)
    sentence = re.sub('CVE-\d{4}-\d{4,7}','[cve]',sentence)
    sentence = sentence.replace('dot', '.')
    sentence = sentence.replace('[.]', '.')
    sentence = re.sub(r"[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}", "[ip]",sentence)
    sentence = re.sub(r"\b^\$2[ayb]\$.{64}$\b",'[hash]',sentence) #hash 512
    sentence = re.sub(r"\b^\$2[ayb]\$.{56}$\b",'[hash]',sentence) #hash 256
    sentence = re.sub(r"\b[a-f0-9]{40}|[A-F0-9]{40}\b",'[hash]',sentence) #hash 1
    sentence = re.sub(r"\b[a-f0-9]{32}|[A-F0-9]{32}\b",'[hash]',sentence) #md5
    sentence = re.sub(r"\b[a-z0-9]+@[a-z]+\.[a-z]{2,3}\b","",sentence)
    sentence = remove_emoji(sentence)
    sentence = chat_words_conversion(sentence)
    #sentence = correct_spellings(sentence)
    sentence = re.sub("(?<=\.)([^.]+)(?:\.(?:co\.uk|ac\.us|com|net|org|info|coop|int|co|ac|ie|co|ai|eu|ca|icu|top|xyz|tk|cn|ga|cf|nl|us|eu|de|hk|am|tv|bingo|blackfriday|gov|edu|mil|arpa|au|ru|[^.]+(?:$|\n)))",'domain',sentence)
    sentence = re.sub("[^a-zA-Z\[\] ']", "", sentence) # removing symbols
    sentence = ' '.join( [w for w in sentence.split() if len(w)>1] )
    sentence = sentence.split()
    word_and_tag = nltk.pos_tag(sentence)
    sentence = [wl.lemmatize(word,pos=get_wordnet_pos(tag)) for word,tag in word_and_tag if word not in set(stopwords.words("english"))]
    sentence = " ".join(sentence)
    tweets.append(sentence)
  return tweets
df['preprocess_text'] = preprocessing_df(df['Text'])

In [ ]:
df

,Unnamed: 0,Unnamed: 0.1,Author ID,User Name,Screen Name,Tweet ID,Created At,Text,Like Count,Quote Count,...,Referenced Tweets,ID of Referenced Tweets,Type of Referenced Tweets,Attachments,Location tagged by the user,Language,Sensitive URL,Reply Settings,Source,preprocess_text
0,2414663,59047,4.501058e+08,ItsTrigger,Trigger,1.344795e+18,2021-01-01 00:00:02+00:00,2020 was rough for us all. I don't usually wri...,7.0,0.0,...,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Twitter Web App,rough u usually write much want open i've part...
1,2414662,59046,2.918316e+08,Paula_Piccard,Paula Piccard,1.344796e+18,2021-01-01 00:00:26+00:00,SolarWinds hackers accessed Microsoft source c...,8.0,0.0,...,NaN,NaN,NaN,{'media_keys': ['3_1344795579990351872']},NaN,en,False,everyone,SmarterQueue,solarwinds hacker access microsoft source code...
2,2414661,59045,5.693841e+08,velez1066,Jorge Velez,1.344796e+18,2021-01-01 00:01:01+00:00,The pattern of #ransomware impacting manufactu...,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Trend Micro,pattern ransomware impact manufacture system c...
3,2414654,59038,9.154360e+17,MoustachedTrex,Johnny,1.344796e+18,2021-01-01 00:03:25+00:00,https://t.co/L4NQPYJ8K7 A list of everything I...,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Twitter Web App,list everything i've experience year
4,2414650,59034,1.059414e+09,An0n_Truth,Anonymous,1.344797e+18,2021-01-01 00:06:06+00:00,The latest The An0nym0us Truth! https://t.co/X...,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Paper.li,late annymus truth thanks itinformationsecurit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430182,50,50,1.384262e+18,hankobaggins,hanko.sol,1.421619e+18,2021-07-31 23:49:53+00:00,@cryptoFLman @BradRSylvain @BoringProtocol FIN...,6.0,0.0,...,[<ReferencedTweet id=1420470148188737542 type=...,1.420470e+18,replied_to,NaN,NaN,en,False,everyone,Twitter Web App,final list sorry delay guy realize twitter api...
430183,48,48,9.503332e+17,HeliosCert,HELIOS CERT,1.421619e+18,2021-07-31 23:50:02+00:00,Sample submitted\n2021-07-31 23:50:02\nDionaea...,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Helios CDC Cert,sample submit dionaea honeypot protocol smbd s...
430184,46,46,2.469117e+08,PMProuk,Jason Normanton 🤍💙💛,1.421619e+18,2021-07-31 23:50:21+00:00,"As cyberattacks skyrocket, Canada needs to wor...",0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,en,False,everyone,TwinyBots,cyberattacks skyrocket canada need work hinder...
430185,38,38,9.202597e+17,cyberinform,Jose Lagdameo Jr,1.421620e+18,2021-07-31 23:52:04+00:00,#Ransomware attacks in #2021 have already surp...,0.0,0.0,...,NaN,NaN,NaN,{'media_keys': ['3_1421619699301892102']},NaN,en,False,everyone,Buffer,ransomware attack already surpass last year in...


In [ ]:
df.to_csv( "/content/drive/MyDrive/output/preprocessed_all_tweets.csv", index=False, encoding='utf-8-sig')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430187 entries, 0 to 430186
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   430187 non-null  int64  
 1   Unnamed: 0.1                 430187 non-null  int64  
 2   Author ID                    430187 non-null  float64
 3   User Name                    430187 non-null  object 
 4   Screen Name                  430160 non-null  object 
 5   Tweet ID                     430187 non-null  float64
 6   Created At                   430187 non-null  object 
 7   Text                         430187 non-null  object 
 8   Like Count                   430187 non-null  float64
 9   Quote Count                  430187 non-null  float64
 10  Reply Count                  430187 non-null  float64
 11  Retweet Count                430187 non-null  float64
 12  Entities                     430173 non-null  object 
 13 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Twitter_Work_Code/output/preprocessed_all_tweets.csv')

In [ ]:
df['preprocess_text'] = df['preprocess_text'].fillna('[NULL]')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430187 entries, 0 to 430186
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   430187 non-null  int64  
 1   Unnamed: 0.1                 430187 non-null  int64  
 2   Author ID                    430187 non-null  float64
 3   User Name                    430187 non-null  object 
 4   Screen Name                  430160 non-null  object 
 5   Tweet ID                     430187 non-null  float64
 6   Created At                   430187 non-null  object 
 7   Text                         430187 non-null  object 
 8   Like Count                   430187 non-null  float64
 9   Quote Count                  430187 non-null  float64
 10  Reply Count                  430187 non-null  float64
 11  Retweet Count                430187 non-null  float64
 12  Entities                     430173 non-null  object 
 13 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [ ]:
MAX_SEQUENCE_LENGTH = 128
max_len=128
MAX_NB_WORDS = 10000
EMBEDDING_DIM=300
VALIDATION_SPLIT = 0.1
traina = True #embeedding layer trainable

In [ ]:
with open('/content/drive/MyDrive/output/rf_classifier', 'rb') as training_model:
    model = pickle.load(training_model)

In [ ]:
with open('/content/drive/MyDrive/output/tfidf', 'rb') as training_model:
    tfidf = pickle.load(training_model)

In [ ]:
# Use `transform` instead of `fit_transform`
X_unseen = tfidf.transform(df['preprocess_text']).toarray()

In [ ]:
# Predict on `X_unseen`
y_pred_unseen = model.predict(X_unseen)

In [ ]:
df['isValid'] = y_pred_unseen

In [ ]:
y_pred_prob= model.predict_proba(X_unseen)

In [ ]:
y_pred_unseen

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
max([y_pred_prob])

array([[0.91540935, 0.08459065],
       [0.97660203, 0.02339797],
       [0.9633487 , 0.0366513 ],
       ...,
       [0.98763014, 0.01236986],
       [0.99342837, 0.00657163],
       [0.93571022, 0.06428978]])

In [ ]:
y_pred_unseen

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
review = 'https://t.co/L4NQPYJ8K7 A list of everything I\'ve experienced this year'
# prepare tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(review)
xtest = tokenizer.texts_to_sequences([review])   #pass array of strings, so []
xtest = pad_sequences(xtest, maxlen=128, padding='post')
xtest

array([[ 2, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int32)

In [ ]:
with open('/content/drive/MyDrive/output/text_classifier', 'rb') as training_model:
    model = pickle.load(training_model)

In [ ]:
with open('/content/drive/MyDrive/output/tokenizer', 'rb') as training_model:
    token = pickle.load(training_model)

In [ ]:
xtest = token.texts_to_sequences(df['preprocess_text'])   #pass array of strings, so []
xtest = pad_sequences(xtest, maxlen=128, padding='post')
xtest

array([[5673,   52, 2800, ...,    0,    0,    0],
       [ 340,   28,  296, ...,    0,    0,    0],
       [2266,    5,  263, ...,    0,    0,    0],
       ...,
       [  38, 2031, 1097, ...,    0,    0,    0],
       [   5,   19,  681, ...,    0,    0,    0],
       [   7,  325,  374, ...,    0,    0,    0]], dtype=int32)

In [ ]:
yhat = model.predict(np.array(xtest))
classes_x=np.argmax(yhat,axis=1)
classes_x

13444/13444 [==============================] - 41s 2ms/step


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
df['isValid']=classes_x

In [ ]:
df[1:50]

,Unnamed: 0,Unnamed: 0.1,Author ID,User Name,Screen Name,Tweet ID,Created At,Text,Like Count,Quote Count,...,ID of Referenced Tweets,Type of Referenced Tweets,Attachments,Location tagged by the user,Language,Sensitive URL,Reply Settings,Source,preprocess_text,isValid
1,2414662,59046,2.918316e+08,Paula_Piccard,Paula Piccard,1.344796e+18,2021-01-01 00:00:26+00:00,SolarWinds hackers accessed Microsoft source c...,8.0,0.0,...,NaN,NaN,{'media_keys': ['3_1344795579990351872']},NaN,en,False,everyone,SmarterQueue,solarwinds hacker access microsoft source code...,0
2,2414661,59045,5.693841e+08,velez1066,Jorge Velez,1.344796e+18,2021-01-01 00:01:01+00:00,The pattern of #ransomware impacting manufactu...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Trend Micro,pattern ransomware impact manufacture system c...,0
3,2414654,59038,9.154360e+17,MoustachedTrex,Johnny,1.344796e+18,2021-01-01 00:03:25+00:00,https://t.co/L4NQPYJ8K7 A list of everything I...,1.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Twitter Web App,list everything i've experience year,0
4,2414650,59034,1.059414e+09,An0n_Truth,Anonymous,1.344797e+18,2021-01-01 00:06:06+00:00,The latest The An0nym0us Truth! https://t.co/X...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Paper.li,late annymus truth thanks itinformationsecurit...,0
5,2414649,59033,2.775447e+08,SolitaryBuffalo,Roger Boss,1.344797e+18,2021-01-01 00:06:19+00:00,"Telehealth poses big #cybersecurity dangers, H...",7.0,0.0,...,NaN,NaN,{'media_keys': ['3_1344797061305294851']},NaN,en,False,everyone,EveryoneSocial,telehealth pose big cybersecurity danger harva...,0
6,2414642,59026,7.693381e+17,CISOmd,rich latayan,1.344798e+18,2021-01-01 00:08:41+00:00,number of #ransomware attacks has jumped by 35...,1.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Twitter for iPhone,number ransomware attack jump percent since av...,0
7,2414640,59024,2.401309e+07,MalwarePatrol,Malware Patrol,1.344798e+18,2021-01-01 00:09:03+00:00,Another active #Phishing targeting Google Driv...,1.0,0.0,...,NaN,NaN,{'media_keys': ['3_1344797749137600514']},NaN,en,False,everyone,MP_Post_New_Malware_Information,another active phishing target google drive ur...,1
8,2414618,59002,1.272531e+18,malware_devil,Malware Devil,1.344799e+18,2021-01-01 00:14:06+00:00,VideoBytes: #brute force attacks increase due ...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,malware_devil_wp,videobytes brute force attack increase due ope...,0
9,2414615,58999,8.783259e+17,LSELabs,Linux Security Labs,1.344799e+18,2021-01-01 00:15:34+00:00,New release: angr 9.0.5262 (binary analysis fr...,2.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,LSE-Labs-queue,new release angr domain,0
10,2414595,58979,1.877962e+08,Robert4787,Robert Morton,1.344801e+18,2021-01-01 00:20:17+00:00,"#Ransomware works, that's the number of attack...",3.0,1.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Twitter for Android,ransomware work that's number attack jump perc...,0


In [ ]:
relevant_df=df.loc[df['isValid'] == 1]

In [ ]:
relevant_df.to_csv( "/content/drive/MyDrive/output/relevant_tweet.csv", index=False, encoding='utf-8-sig')

In [ ]:
relevant_df

,Unnamed: 0,Unnamed: 0.1,Author ID,User Name,Screen Name,Tweet ID,Created At,Text,Like Count,Quote Count,...,ID of Referenced Tweets,Type of Referenced Tweets,Attachments,Location tagged by the user,Language,Sensitive URL,Reply Settings,Source,preprocess_text,isValid
7,2414640,59024,2.401309e+07,MalwarePatrol,Malware Patrol,1.344798e+18,2021-01-01 00:09:03+00:00,Another active #Phishing targeting Google Driv...,1.0,0.0,...,NaN,NaN,{'media_keys': ['3_1344797749137600514']},NaN,en,False,everyone,MP_Post_New_Malware_Information,another active phishing target google drive ur...,1
17,2414566,58950,3.973558e+09,PhishStats,Phish.Stats 🐟,1.344802e+18,2021-01-01 00:26:26+00:00,new #phishing at hXXp://departmen-advant-page[...,2.0,1.0,...,NaN,NaN,{'media_keys': ['3_1344802123939278852']},NaN,en,False,everyone,PhishStats,new phishing [defanged] [ip] united state alch...,1
20,2414553,58937,1.944418e+07,ecarlesi,Emiliano Carlesi,1.344803e+18,2021-01-01 00:31:00+00:00,Possible threat on hxxps://teamfrenette[.]com/...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,PhishingPicker,possible threat [defanged] phishing opendir,1
21,2414545,58929,1.944418e+07,ecarlesi,Emiliano Carlesi,1.344803e+18,2021-01-01 00:31:25+00:00,Possible threat on hxxps://teamfrenette[.]com/...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,PhishingPicker,possible threat [defanged] phishing opendir,1
77,2414333,58717,1.944418e+07,ecarlesi,Emiliano Carlesi,1.344825e+18,2021-01-01 01:58:06+00:00,Threat on hxxp://fraudcheck-barclays[.]info/Ac...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,PhishingPicker,threat [defanged] phishing,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430154,306,306,5.789760e+08,phishunt_io,phishunt.io,1.421606e+18,2021-07-31 22:56:45+00:00,#NewPhishing | #phishing #scam\n\n🌐 /bankofame...,1.0,0.0,...,NaN,NaN,{'media_keys': ['3_1421605781359583235']},NaN,en,False,everyone,phishunt,newphishing phishing scam bankofamericamiamico...,1
430161,264,264,2.504830e+09,BethkazV,Beth Kazakewich,1.421608e+18,2021-07-31 23:06:42+00:00,Test your knowledge at https://t.co/yvg8zciMOZ...,1.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Twitter Web App,test knowledge eset antivirus phishing contest...,1
430165,234,234,1.207782e+18,Coug2Coug,Coug2CougPodcast,1.421610e+18,2021-07-31 23:14:20+00:00,@brookechesney @EasopWinston Go Cougs #CvE,0.0,0.0,...,1.421608e+18,replied_to,NaN,NaN,en,False,everyone,Twitter for iPhone,go cougs cve,1
430175,115,115,1.944418e+07,ecarlesi,Emiliano Carlesi,1.421616e+18,2021-07-31 23:37:40+00:00,Threat on hxxp://certiport[.]games/postale[.]z...,0.0,0.0,...,NaN,NaN,"{'media_keys': ['3_1421616066598522880', '3_14...",NaN,en,False,everyone,PhishingPicker,threat [defanged] phishing opendir namecom,1


In [ ]:
irrelevant_df=df.loc[df['isValid'] == 0]

In [ ]:
irrelevant_df

,Unnamed: 0,Unnamed: 0.1,Author ID,User Name,Screen Name,Tweet ID,Created At,Text,Like Count,Quote Count,...,ID of Referenced Tweets,Type of Referenced Tweets,Attachments,Location tagged by the user,Language,Sensitive URL,Reply Settings,Source,preprocess_text,isValid
0,2414663,59047,4.501058e+08,ItsTrigger,Trigger,1.344795e+18,2021-01-01 00:00:02+00:00,2020 was rough for us all. I don't usually wri...,7.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Twitter Web App,rough u usually write much want open i've part...,0
1,2414662,59046,2.918316e+08,Paula_Piccard,Paula Piccard,1.344796e+18,2021-01-01 00:00:26+00:00,SolarWinds hackers accessed Microsoft source c...,8.0,0.0,...,NaN,NaN,{'media_keys': ['3_1344795579990351872']},NaN,en,False,everyone,SmarterQueue,solarwinds hacker access microsoft source code...,0
2,2414661,59045,5.693841e+08,velez1066,Jorge Velez,1.344796e+18,2021-01-01 00:01:01+00:00,The pattern of #ransomware impacting manufactu...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Trend Micro,pattern ransomware impact manufacture system c...,0
3,2414654,59038,9.154360e+17,MoustachedTrex,Johnny,1.344796e+18,2021-01-01 00:03:25+00:00,https://t.co/L4NQPYJ8K7 A list of everything I...,1.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Twitter Web App,list everything i've experience year,0
4,2414650,59034,1.059414e+09,An0n_Truth,Anonymous,1.344797e+18,2021-01-01 00:06:06+00:00,The latest The An0nym0us Truth! https://t.co/X...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Paper.li,late annymus truth thanks itinformationsecurit...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430181,58,58,1.276174e+18,cybermaterial_,✇ ⚪🛡 Cyb≡rMaterial ✪,1.421619e+18,2021-07-31 23:48:19+00:00,NEW POST 📢 Types of Malware - #cybersecurity #...,4.0,0.0,...,NaN,NaN,{'media_keys': ['3_1421618758175629313']},NaN,en,False,everyone,CyberMaterial,new post type malware cybersecurity ransomware...,0
430182,50,50,1.384262e+18,hankobaggins,hanko.sol,1.421619e+18,2021-07-31 23:49:53+00:00,@cryptoFLman @BradRSylvain @BoringProtocol FIN...,6.0,0.0,...,1.420470e+18,replied_to,NaN,NaN,en,False,everyone,Twitter Web App,final list sorry delay guy realize twitter api...,0
430184,46,46,2.469117e+08,PMProuk,Jason Normanton 🤍💙💛,1.421619e+18,2021-07-31 23:50:21+00:00,"As cyberattacks skyrocket, Canada needs to wor...",0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,TwinyBots,cyberattacks skyrocket canada need work hinder...,0
430185,38,38,9.202597e+17,cyberinform,Jose Lagdameo Jr,1.421620e+18,2021-07-31 23:52:04+00:00,#Ransomware attacks in #2021 have already surp...,0.0,0.0,...,NaN,NaN,{'media_keys': ['3_1421619699301892102']},NaN,en,False,everyone,Buffer,ransomware attack already surpass last year in...,0


In [ ]:
irrelevant_df.to_csv( "/content/drive/MyDrive/output/irrelevant_tweet.csv", index=False, encoding='utf-8-sig')

In [ ]:
new_df = df.loc[df['isValid'] == 0]

In [ ]:
part_df = new_df[40000:60000]

In [ ]:
part_df.to_csv( "/content/drive/MyDrive/output/irrelevant_tweet_part_3.csv", index=False, encoding='utf-8-sig')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(text), y, test_size=1, stratify=y,random_state=100)


In [ ]:
x_test

array([[   43,  6844,   489,  2625,   831, 51857,   903,    60,   487,
         1594, 36450,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [ ]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 26.5 MB/s 
     |████████████████████████████████| 1.6 MB 52.8 MB/s 


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
    keras_model = keras.Sequential()
    #keras_model.add(word2vec_model.wv.get_keras_embedding(True))
    keras_model.add(Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=traina))
    keras_model.add(Dropout(0.5))
    keras_model.add(Conv1D(filters=hp.Int('filters',
                                        min_value=32,
                                        max_value=512,
                                        step = 32),
                kernel_size=hp.Int('kernel_size',
                                        min_value=3,
                                        max_value=11,
                                        step = 2),
                padding='same',
                activation='relu'))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.5))
    keras_model.add(Conv1D(filters=hp.Int('filters',
                                        min_value=32,
                                        max_value=512,
                                        step = 32),
                kernel_size=hp.Int('kernel_size',
                                        min_value=3,
                                        max_value=11,
                                        step = 2),
                padding='same',
                activation='relu'))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.5))
    keras_model.add(Conv1D(filters=hp.Int('filters',
                                        min_value=32,
                                        max_value=512,
                                        step = 32),
                kernel_size=hp.Int('kernel_size',
                                        min_value=3,
                                        max_value=11,
                                        step = 2),
                padding='same',
                activation='relu'))
    keras_model.add(GlobalMaxPool1D())
    keras_model.add(layers.Flatten())
    keras_model.add(Dropout(0.5))
    keras_model.add(Dense(units=hp.Int('units',
                                        min_value=64,
                                        max_value=256,
                                        step=32),
                           activation='relu'))
    keras_model.add(Dropout(0.5))
    keras_model.add(Dense(2))
    keras_model.add(Activation('softmax'))
    keras_model.compile(optimizer=keras.optimizers.Adam(
        hp.Choice('learning_rate',
                  values=[1e-2, 1e-3, 1e-4])),
    loss='binary_crossentropy',
    metrics=['accuracy'])
    return keras_model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='/content/drive/MyDrive/CNN_tuner',
    project_name='1042')
tuner.search_space_summary()

Search space summary
Default search space size: 4
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
kernel_size (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 11, 'step': 2, 'sampling': None}
units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
tuner.search(x_train, y_train,
             epochs=10,
             validation_data=(x_val, y_val))

models = tuner.get_best_models(num_models=2)

Trial 1 Complete [00h 12m 19s]
val_loss: 0.5588799317677816

Best val_loss So Far: 0.5588799317677816
Total elapsed time: 00h 12m 19s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
160               |288               |filters
7                 |3                 |kernel_size
64                |160               |units
0.0001            |0.0001            |learning_rate

Epoch 1/10
1289/1289 [==============================] - 23s 17ms/step - loss: 0.6994 - accuracy: 0.5497 - val_loss: 0.6901 - val_accuracy: 0.5655
Epoch 2/10
1289/1289 [==============================] - 21s 16ms/step - loss: 0.6854 - accuracy: 0.5643 - val_loss: 0.6777 - val_accuracy: 0.5655
Epoch 3/10
1289/1289 [==============================] - 20s 16ms/step - loss: 0.6676 - accuracy: 0.5754 - val_loss: 0.6575 - val_accuracy: 0.6144
Epoch 4/10
1289/1289 [==============================] - 21s 16ms/step - loss: 0.6459 - accuracy: 0.6110 - val_loss: 0.6331 - val_accuracy: 0.6443
Epoch 5/1

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size,
                        EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=traina))
model.add(Conv1D(288,3,padding='same',activation='relu'))
model.add(MaxPool1D())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(160, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
keras_model.fit(x_train, y_train, batch_size=16, epochs=1,  validation_data=(x_val,y_val))

Epoch 1/50
905/905 [==============================] - 161s 175ms/step - loss: 0.1724 - accuracy: 0.9197 - val_loss: 0.0609 - val_accuracy: 0.9829
Epoch 2/50
905/905 [==============================] - 124s 137ms/step - loss: 0.0651 - accuracy: 0.9793 - val_loss: 0.0521 - val_accuracy: 0.9851
Epoch 3/50
905/905 [==============================] - 124s 137ms/step - loss: 0.0482 - accuracy: 0.9854 - val_loss: 0.0512 - val_accuracy: 0.9856
Epoch 4/50
905/905 [==============================] - 124s 137ms/step - loss: 0.0414 - accuracy: 0.9886 - val_loss: 0.0512 - val_accuracy: 0.9873
Epoch 5/50
905/905 [==============================] - 125s 138ms/step - loss: 0.0344 - accuracy: 0.9902 - val_loss: 0.0564 - val_accuracy: 0.9851
Epoch 6/50
905/905 [==============================] - 119s 132ms/step - loss: 0.0272 - accuracy: 0.9921 - val_loss: 0.0599 - val_accuracy: 0.9851
Epoch 7/50
905/905 [==============================] - 118s 131ms/step - loss: 0.0240 - accuracy: 0.9929 - val_loss: 0.0573 -

In [ ]:
y_test = np.argmax(y_test, axis=1)
y_pred_enc = model.predict(x_test)
y_pred_enc = np.argmax(y_pred_enc, axis=1)

169/169 [==============================] - 0s 1ms/step


In [ ]:
print(classification_report(y_test,y_pred_enc))

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
seed = 42
n_splits=10
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

In [ ]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1

In [ ]:
texts = np.array(text)

In [ ]:
y=df['isValid']
y=np.array(y)

In [ ]:
y

array([1, 1, 0, ..., 1, 0, 1])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

In [ ]:
checkpointer = ModelCheckpoint('CNN' + "_weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor="val_loss", verbose=1,
                               save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', mode='auto')
tensorboard_cnn = TensorBoard(log_dir='./Graph_CNN', histogram_freq=0,write_graph=True, write_images=True)

In [ ]:
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, f1_score, auc,
                             plot_confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, 
                             precision_recall_curve)

In [ ]:
# Accuracy scores lists
acc_scores, prec_scores, recall_scores, f1_scores, fold = [], [], [], [], []
for train, test in kfold.split(texts, y):
  x_train=texts[train]
  y_train=y[train]
  x_test=texts[test]
  y_test=y[test]
  le = preprocessing.LabelEncoder()
  y_train = le.fit_transform(y_train)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test) 
  model = Sequential()
  model.add(Embedding(vocab_size,
                          EMBEDDING_DIM,
                          weights=[embedding_matrix],
                          input_length=MAX_SEQUENCE_LENGTH,
                          trainable=traina))
  model.add(Conv1D(160,7,padding='same',activation='relu'))
  model.add(MaxPool1D())
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(96, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation='sigmoid'))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])
  model.fit(x_train, y_train, batch_size=16, epochs=10, validation_split=0.1)
  
  pred = model.predict(x_test)
  pred=np.argmax(pred, axis=1)
  ytest=np.argmax(y_test, axis=1)
  print(classification_report(ytest,pred))
  fig, ax = plt.subplots(figsize=(7, 7))
  sns.heatmap(confusion_matrix(ytest, pred), annot=True, 
              cbar=False, fmt='1d', cmap='Blues', ax=ax)
  ax.set_title('Confusion Matrix', loc='left', fontsize=16)
  ax.set_xlabel('Predicted')
  ax.set_ylabel('Actual')
  plt.show()
  #acc_scores=acc_scores.append(accuracy_score(ytest,pred))
  #f1_scores=f1_scores.append(f1_score(y_test,pred,average="weighted"))
  # Increase fold number
  fold_no = fold_no + 1
print("-"*30)
print(f'Accuracy:  {np.mean(acc_scores):.5f} +- {np.std(acc_scores):5f}')
print(f'F1 Score:  {np.mean(f1_scores):.5f} +- {np.std(f1_scores):5f}')
#print(f'ROC AUC:   {np.mean(roc_auc_scores):.5f} +- {np.std(roc_auc_scores):5f}')
print("")


In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')